In [3]:
import jqdatasdk as jq
import pandas as pd
import numpy as np

In [ ]:
jq.auth('18436106390','MDLmyf123.') 

In [ ]:
jq.get_query_count()

## 读取Fund Main表

In [ ]:
fund_main = jq.finance.run_query(jq.query(jq.finance.FUND_MAIN_INFO))

In [ ]:
fund_main.info()

In [ ]:
fund_main['start_date']= pd.to_datetime(fund_main['start_date'])
fund_main['end_date']= pd.to_datetime(fund_main['end_date'])

In [ ]:
fund_main = fund_main.drop(['id','statistics_main_code','invest_style_id','invest_style'],axis=1)

In [ ]:
fund_main.info()

In [ ]:
import sqlite3

conn = sqlite3.connect('db.sqlite3')

In [ ]:
fund_main

In [ ]:
sql = "delete from funds_fundmain"
conn.execute(sql)

In [ ]:
conn.commit()

In [ ]:
fund_main.to_sql('funds_fundmain',conn,if_exists='append',index=False)

## 读取基金净值数据

读取前100个基金的数据

In [ ]:
fund_main.main_code[:100]

读取净值数据

In [ ]:
fund_net_value = jq.finance.run_query(jq.query(jq.finance.FUND_NET_VALUE).filter(jq.finance.FUND_NET_VALUE.code==fund_main.main_code[0],jq.finance.FUND_NET_VALUE.day>'2010-01-01'))

In [ ]:
fund_net_value.info()

In [ ]:
conn.execute('delete from funds_fundsnetvalue')

In [ ]:
conn.commit()

In [ ]:
for fund_id in fund_main.main_code[:100]:
    #读取数据
    fund_netvalue = jq.finance.run_query(jq.query(jq.finance.FUND_NET_VALUE).
                     filter(jq.finance.FUND_NET_VALUE.code==fund_id,jq.finance.FUND_NET_VALUE.day>='2010-01-01'))
    #修改字段名
    fund_netvalue['code_id'] = fund_netvalue['code']
    #删除id和code 
    fund_netvalue = fund_netvalue.drop(['id','code'],axis=1)
    #day字段转换日期格式
    fund_netvalue['day']= pd.to_datetime(fund_netvalue['day'])
    #计算日收益率
    fund_netvalue['daily_return'] = np.log(fund_netvalue['sum_value']/fund_netvalue['sum_value'].shift(1))*100
    #计算月收益率
    fund_netvalue['month_return'] = np.log(fund_netvalue['sum_value']/fund_netvalue['sum_value'].shift(22))*100
    #计算季度收益率
    fund_netvalue['quarter_return'] = np.log(fund_netvalue['sum_value']/fund_netvalue['sum_value'].shift(64))*100
    #计算累计收益率
    fund_netvalue['total_return'] = np.log(fund_netvalue['sum_value']/fund_netvalue['sum_value'][0])*100
    #保存
    fund_netvalue.to_sql('funds_fundsnetvalue',conn, if_exists='append',index=False)


In [ ]:
sql = "select * from funds_fundsnetvalue where day='2021-11-11'"
myfunds = conn.execute(sql)

In [ ]:
values = myfunds.fetchall()

In [ ]:
sql = "select fund.name, fund.main_code, netvalue.day,netvalue.net_value,netvalue.sum_value,netvalue.daily_return,netvalue.month_return, netvalue.quarter_return,netvalue.total_return from funds_fundmain as fund, funds_fundsnetvalue as netvalue where fund.main_code == netvalue.code_id and netvalue.day ='2021-11-11 00:00:00'"

In [ ]:
fundslist = pd.read_sql(sql,conn)

In [ ]:
fundslist

In [ ]:
sql2 = "select fund.name, netvalue.day,netvalue.net_value,netvalue.sum_value,netvalue.daily_return,netvalue.month_return, netvalue.quarter_return,netvalue.total_return from funds_fundmain as fund, funds_fundsnetvalue as netvalue where fund.main_code == netvalue.code_id and fund.main_code ='000001'"

fundslist = pd.read_sql(sql2,conn)

fundslist

在推荐表中插入数据

1. 泰和混合，长跑冠军 精选优质企业 +161.76% 近五年涨幅
2. 新能源， 产业动力 新能源先锋掌舵 +347.38% 近五年涨幅
3. 沪深300， 投资A股核心代表性指数 +50.22% 近五年涨幅
4. 稳固收益，固收打底 权益增强 +38.18% 近五年涨幅

In [ ]:
recomm = pd.DataFrame([['000595','泰和混合','长跑冠军 精选优质企业',161.76],
                       ['003984','新能源','产业动力 新能源先锋掌舵',347.38],
                       ['160706','沪深300','投资A股核心代表性指数',50.22],
                       ['070020','稳固收益','固收打底 权益增强',38.18]],columns=['code','name','comment','total_return'])

In [ ]:
recomm

In [ ]:
recomm.to_sql('funds_recommendfunds',conn,if_exists='append',index=False)